In [ ]:
# %pip install -r requirements.txt

In [ ]:
# !huggingface-cli download lmsys/vicuna-7b-v1.3 --local-dir '/root/SD-research/base_models/vicuna'
# !huggingface-cli download yuhuili/EAGLE-Vicuna-7B-v1.3 --local-dir '/root/SD-research/ea_models/vicuna'

In [ ]:
# !huggingface-cli download meta-llama/Llama-2-7b-chat-hf --local-dir '/root/SD-research/base_models/llama2'
# !huggingface-cli download yuhuili/EAGLE-llama2-chat-7B --local-dir '/root/SD-research/ea_models/llama2'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 16 files: 100%|██████████████████████| 16/16 [00:00<00:00, 5904.87it/s]
/root/SD-research/base_models/llama2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 4 files:   0%|                                   | 0/4 [00:00<?, ?it/s]Downloading 'README.md' to '/root/SD-research/ea_models/llama2/.cache/huggingface/download/README.md.154df8298fab5ecf322016157858e08cd1bccbe1.incomplete'

README.md: 100%|██████████████████████████████| 28.0/28.0 [00:00<00:00, 263kB/s]
Download complete. Moving file to /root/SD-research/ea_models/llama2/README.md

.gitattributes: 100%|██████████████████████| 1.52k/1.52k [00:00<00:00, 22.7MB/s]
Download complete. Moving file to /root/SD-research/ea_models/llama2/.gitattributes
Fetching 4 files:  25%|██████▊                    | 1/4 [00:00<00:01,  1.97it/s]Downloading 'pytorch_model.bin' to '/root/SD-research/ea_models/llama2/.cache/huggingface/download/pytorch_model.bin.3f1e4f6bef7befeba9d08754f98088799d754250b20fc61a46081e8ea0ca978b.incomplete'

config.json: 100%|█████████████████████████████| 560/560 [00:00<00:00, 4.48MB/s]
Download complete. Moving file to /root/SD-research/ea_models/llama2/config.json
Fe

In [30]:
import torch
import argparse
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import json
import csv
from pathlib import Path
import logging
from datetime import datetime
from typing import List, Dict
import yaml
import os
from eagle.model.ea_model import EaModel
from fastchat.model import get_conversation_template
import time
from statistics import mean
import math

In [3]:
model = EaModel.from_pretrained(
    base_model_path='/root/SD-research/base_models/vicuna',
    ea_model_path='/root/SD-research/ea_models/vicuna',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)
model.eval() 

LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.08s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokeni

EaModel(
  (base_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096, padding_idx=0)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm)

In [ ]:
def save_results(results: List[Dict], 
                 output_filename: str, 
                 config_file: str, 
                 config: Dict,
                 output_dir = None,
                 avg_acceptance_rate=None,
                 avg_token_sec = None,
                 total_rejected_token_contexts=None,
                 final_latency_results=None):
    """Save results to CSV and JSON files, including config file reference."""
    # Extract the base name of the config file (e.g., "config.yaml" -> "config")
    config_name = Path(config_file).stem
    timestamp = datetime.now().strftime("%m%d_%H%M")
    
    # File paths with config file name
    json_path = output_dir / f"{output_filename}_{config_name}_{timestamp}.json"
    clean_json_path = output_dir / f"clean_{output_filename}_{config_name}_{timestamp}.json"
    
    # Save results and configuration as JSON
    results_with_config = {
        'exp_name': output_filename,
        'average_token_per_sec': avg_token_sec,
        'average_acceptance_rate': avg_acceptance_rate,
        'average_latency_by_phase': final_latency_results,
        'total_rejected_token_contexts': total_rejected_token_contexts,
        "config": config,  # Embed the configuration in the JSON
        "results": results
    }
    
    clean_results = {
        'exp_name': output_filename,
        'average_token_per_sec': avg_token_sec,
        'average_acceptance_rate': avg_acceptance_rate,
        'average_latency_by_phase': final_latency_results,
        'total_rejected_token_contexts': total_rejected_token_contexts
    }
    
    with open(clean_json_path, 'w') as f:
        json.dump(clean_results, f, indent=2)

    with open(json_path, 'w') as f:
        json.dump(results_with_config, f, indent=2)

    # logger.info(f"Results saved to {json_path}")


In [4]:
def truncate_list(lst, num):
    if num not in lst:
        return lst
    first_index = lst.index(num)
    return lst[:first_index + 1]

In [36]:
timestamp = datetime.now().strftime("%m%d_%H%M")
results = []
for i in tqdm(range(4)):
    prompts = []    
    results = []
    # load input data
    segment_string = f'{500*i}~{500*(i+1)}'
    input_filename = f"../data/ArXiv/4K/ArXiv_4K_{segment_string}.json"
    exp_name = f"EAGLE_vicuna_7b_v1.3_{segment_string}.json"
    with open(input_filename, 'r', encoding='utf-8') as f:
        raw_input = json.load(f)
        input_data = [item['final_input'] for item in raw_input]
        temperature = 0.0        
        tps_eas = []
        tps_ars = []
        ea_ar_speedups = []
        for item in input_data :
            conv = get_conversation_template("vicuna")
            conv.append_message(conv.roles[0], item)
            conv.append_message(conv.roles[1], None)        
            prompt = conv.get_prompt()
                        
            input_ids = model.tokenizer([prompt]).input_ids
            input_ids = torch.as_tensor(input_ids).cuda()
            input_len = input_ids.shape[1]
            naive_text = []
            cu_len = input_len
            totaltime=0
            start_time=time.time()
            total_ids=0
            
            for output_ids in model.ea_generate(input_ids, temperature=temperature, max_steps=512):
                totaltime+=(time.time()-start_time)
                total_ids+=1
                decode_ids = output_ids[0, input_len:].tolist()
                decode_ids = truncate_list(decode_ids, model.tokenizer.eos_token_id)
                text = model.tokenizer.decode(decode_ids, skip_special_tokens=True, spaces_between_special_tokens=False,
                                            clean_up_tokenization_spaces=True, )
                # naive_text.append(model.tokenizer.decode(output_ids[0, cu_len], skip_special_tokens=True,
                #                                         spaces_between_special_tokens=False,
                #                                         clean_up_tokenization_spaces=True, ))

                cu_len = output_ids.shape[1]
                new_tokens = cu_len-input_len
                start_time = time.time()   
                
            tps_ea = new_tokens/totaltime
            tps_eas.append(tps_ea)       
                        
            for output_ids in model.naive_generate(input_ids, temperature=temperature, max_steps=512):
                totaltime += (time.time() - start_time)
                total_ids+=1
                decode_ids = output_ids[0, input_len:].tolist()
                decode_ids = truncate_list(decode_ids, model.tokenizer.eos_token_id)
                text = model.tokenizer.decode(decode_ids, skip_special_tokens=True, spaces_between_special_tokens=False,
                                            clean_up_tokenization_spaces=True, )
                # naive_text.append(model.tokenizer.decode(output_ids[0, cu_len], skip_special_tokens=True,
                #                                         spaces_between_special_tokens=False,
                #                                         clean_up_tokenization_spaces=True, ))
                cu_len = output_ids.shape[1]
                new_tokens = cu_len - input_len
                tps_ars.append(new_tokens/totaltime)            
                start_time = time.time()
                
            tps_ar = new_tokens/totaltime
            tps_ars.append(tps_ar)       
            
            ea_ar_speedups.append(tps_ea/tps_ar)
            # print(ea_ar_speedups)
    tps_ar_avg = mean(tps_ars)
    tps_ea_avg = mean(tps_eas)
    speedup_avg = mean(ea_ar_speedups)
    
    print(tps_ar_avg, tps_ea_avg, speedup_avg)
    
    results.append(
        {
        "exp_name": exp_name,
        "avg_speedup_ratio": round(speedup_avg, 3),
        "avg_token_per_sec_eagle": round(tps_ea_avg, 3),
        "avg_token_per_sec_ar": round(tps_ar_avg, 3)
        }        
    )
    
    
    
output_file = "../results/EAGLE/vicuna-7B-v1.3/FINAL_result_summary.json"

with open(output_file, "w") as f:
    json.dump(results, f, indent=2)

 25%|██▌       | 1/4 [12:49<38:28, 769.57s/it]

17.37766400522225 92.35047840227244 3.781207772054285


 50%|█████     | 2/4 [27:39<28:00, 840.09s/it]

16.782976542932154 86.19873214397346 3.632136237680685


 75%|███████▌  | 3/4 [44:04<15:06, 906.44s/it]

16.631083731703512 84.84719936319895 3.5910582458725457


100%|██████████| 4/4 [52:42<00:00, 790.59s/it]

15.217228808676397 61.05481053008887 4.069666688159131


In [ ]:
timestamp = datetime.now().strftime("%m%d_%H%M")
results = []
for i in tqdm(range(4)):
    prompts = []    
    # load input data
    segment_string = f'{500*i}~{500*(i+1)}'
    input_filename = f"../data/ArXiv/4K/ArXiv_4K_{segment_string}.json"
    exp_name = f"EAGLE_vicuna_7b_v1.3_{segment_string}.json"
    with open(input_filename, 'r', encoding='utf-8') as f:
        raw_input = json.load(f)
        input_data = [item['final_input'] for item in raw_input]
        temperature = 0.0        
        tps_eas = []
        tps_ars = []    
        total_tps_ea = 0.0
        total_tps_ar = 0.0
        ea_ar_speedups = []
        for item in input_data :
            conv = get_conversation_template("vicuna")
            conv.append_message(conv.roles[0], item)
            conv.append_message(conv.roles[1], None)        
            prompt = conv.get_prompt()
                        
            input_ids = model.tokenizer([prompt]).input_ids
            input_ids = torch.as_tensor(input_ids).cuda()
            input_len = input_ids.shape[1]
            naive_text = []
            cu_len = input_len
            totaltime=0
            start_time=time.time()
            # total_ids=0
            
            for output_ids in model.ea_generate(input_ids, temperature=temperature, max_steps=512):
                totaltime+=(time.time()-start_time)
                total_ids+=1
                decode_ids = output_ids[0, input_len:].tolist()
                decode_ids = truncate_list(decode_ids, model.tokenizer.eos_token_id)
                text = model.tokenizer.decode(decode_ids, skip_special_tokens=True, spaces_between_special_tokens=False,
                                            clean_up_tokenization_spaces=True, )
                # naive_text.append(model.tokenizer.decode(output_ids[0, cu_len], skip_special_tokens=True,
                #                                         spaces_between_special_tokens=False,
                #                                         clean_up_tokenization_spaces=True, ))

                cu_len = output_ids.shape[1]
                new_tokens = cu_len-input_len
                start_time = time.time()   
                
            tps_ea = new_tokens/totaltime
            total_tps_ea += tps_ea
                        
            for output_ids in model.naive_generate(input_ids, temperature=temperature, max_steps=512):
                totaltime += (time.time() - start_time)
                total_ids+=1
                decode_ids = output_ids[0, input_len:].tolist()
                decode_ids = truncate_list(decode_ids, model.tokenizer.eos_token_id)
                text = model.tokenizer.decode(decode_ids, skip_special_tokens=True, spaces_between_special_tokens=False,
                                            clean_up_tokenization_spaces=True, )
                # naive_text.append(model.tokenizer.decode(output_ids[0, cu_len], skip_special_tokens=True,
                #                                         spaces_between_special_tokens=False,
                #                                         clean_up_tokenization_spaces=True, ))
                cu_len = output_ids.shape[1]
                new_tokens = cu_len - input_len
                tps_ars.append(new_tokens/totaltime)            
                start_time = time.time()
                
            tps_ar = new_tokens/totaltime
            total_tps_ar += tps_ar
            # print(ea_ar_speedups)
    # tps_ar_avg = mean(tps_ars)
    # tps_ea_avg = mean(tps_eas)
    # speedup_avg = mean(ea_ar_speedups)
    
    tps_ea_avg = total_tps_ea / len(input_data) # 80
    tps_ar_avg = total_tps_ar / len(input_data) # 80
    speedup_avg = tps_ea_avg / tps_ar_avg
    
    print(tps_ar_avg, tps_ea_avg, speedup_avg)
    
    results.append(
        {
        "exp_name": exp_name,
        "avg_speedup_ratio": round(speedup_avg, 3),
        "avg_token_per_sec_eagle": round(tps_ea_avg, 3),
        "avg_token_per_sec_ar": round(tps_ar_avg, 3)
        }        
    )   
    
output_file = "../results/EAGLE/vicuna-7B-v1.3/FINAL_result_summary_novar.json"

with open(output_file, "w") as f:
    json.dump(results, f, indent=2)

[{'exp_name': 'EAGLE_vicuna_7b_v1.3_1500~2000.json',
  'avg_speedup_ratio': 4.07,
  'avg_token_per_sec_eagle': 61.055,
  'avg_token_per_sec_ar': 15.217}]

In [62]:
import gc
gc.collect()

12739

In [39]:
i = 1
segment_string = f'{500*i}~{500*(i+1)}'
input_filename = f"../data/ArXiv/4K/ArXiv_4K_{segment_string}.json"
with open(input_filename, 'r', encoding='utf-8') as f:
    raw_input = json.load(f)
    input_data = [item['final_input'] for item in raw_input]        
    print(input_data[0])

conv = get_conversation_template("vicuna")
conv.append_message(conv.roles[0], input_data[0])
conv.append_message(conv.roles[1],None)
prompt = conv.get_prompt()
prompt

You will receive the full text of a scientific paper. Your task is to write a formal abstract (under 300 words) that includes:

- The primary objective or research question.
- A brief description of the methodology.
- The main findings or results.
- The significance or implications of the findings.

Use clear, professional, and concise language suitable for publication.

### Paper content

there are several reasons to believe that a population of intergalactic globular clusters ( igcs ) should exist outside of galaxies :    \(1 ) the jeans mass at recombination was @xmath0 solar masses , and hence globular cluster sized objects could have formed wherever the local density of matter was high enough . \(2 ) many galaxies may have met their demise over a hubble time as a result of collisions and tidal disruption . globular clusters are likely to survive the disruption of their parent galaxy , resulting in the gradual accumulation of a population of igcs . intergalactic stars , planetary n

"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: You will receive the full text of a scientific paper. Your task is to write a formal abstract (under 300 words) that includes:\n\n- The primary objective or research question.\n- A brief description of the methodology.\n- The main findings or results.\n- The significance or implications of the findings.\n\nUse clear, professional, and concise language suitable for publication.\n\n### Paper content\n\nthere are several reasons to believe that a population of intergalactic globular clusters ( igcs ) should exist outside of galaxies :    \\(1 ) the jeans mass at recombination was @xmath0 solar masses , and hence globular cluster sized objects could have formed wherever the local density of matter was high enough . \\(2 ) many galaxies may have met their demise over a hubble time as a result of collisions and tidal disruption . gl

In [ ]:
input_ids=model.tokenizer([prompt]).input_ids
input_ids = torch.as_tensor(input_ids).cuda()
output_ids=model.eagenerate(input_ids,temperature=0.5,max_new_tokens=512)
output=model.tokenizer.decode(output_ids[0])

In [41]:
output_ids

tensor([[    1,   319, 13563,  ...,  6757, 29889,     2]], device='cuda:0')

In [42]:
prompt

"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: You will receive the full text of a scientific paper. Your task is to write a formal abstract (under 300 words) that includes:\n\n- The primary objective or research question.\n- A brief description of the methodology.\n- The main findings or results.\n- The significance or implications of the findings.\n\nUse clear, professional, and concise language suitable for publication.\n\n### Paper content\n\nthere are several reasons to believe that a population of intergalactic globular clusters ( igcs ) should exist outside of galaxies :    \\(1 ) the jeans mass at recombination was @xmath0 solar masses , and hence globular cluster sized objects could have formed wherever the local density of matter was high enough . \\(2 ) many galaxies may have met their demise over a hubble time as a result of collisions and tidal disruption . gl

In [47]:
import re
re.split('ASSISTANT: ',output)[-1]


'The primary objective of this study was to investigate the existence of intergalactic globular clusters (IGCs) outside of galaxies. The research question was whether IGCs exist and could be detected in the field of the rich galaxy cluster A1185. The methodology involved acquiring deep, multicolored images of the A1185 field using the Hubble Space Telescope (HST) and the new Advanced Camera for Surveys (ACS). The authors aimed to detect the peak of the assumed universal Gaussian-like globular cluster luminosity function and use color information to infer the metallicities of the IGCs. The main findings were that the authors were able to reach sufficiently faint magnitudes to reliably detect the luminosity function turnover. Preliminary analysis of the data revealed that the number and colors of the IGCs will provide constraints on the number and types of galaxies that have been destroyed or stripped over a Hubble time. The significance of the findings is that the existence of IGCs migh